# **VSC 에서 OpenAI Whisper 사용하기**



---



- 💡 **NOTE**
    - 이 과정은 PC Visual Studio Code(이하 VSC)에서 진행합니다.
    - 가상환경 생성과 라이브러리 설치는 VSC 터미널에서 진행합니다.
    - 아래 내용은 Windows PC 환경을 가정하여 설명되었습니다.    
    - 참고 : https://bcuts.tistory.com/192





---



## **Whisper 소개**
- Whisper는 OpenAI가 개발한 자동 음성 인식(ASR: Automatic Speech Recognition) 시스템
- 2022년 9월 OpenAI에 의해 처음 공개한 오픈 소스 모델
- 인터넷에서 수집한 방대한 양의 오디오 데이터로 훈련되어, 다양한 언어와 악센트, 배경 소음 속에서도 매우 뛰어난 정확도를 자랑함
- 기존의 음성 인식 모델들이 가진 한계를 극복하고, 음성-텍스트 변환과 언어 번역을 동시에 수행할 수 있도록 설계된 혁신적인 모델
- 대규모 약한 지도 학습(Large-Scale Weak Supervision)"이라는 독특한 방식을 사용
    - Whisper는 인터넷에서 수집한 방대한 양의 오디오와 해당 오디오의 전사본으로 학습
    - 이 데이터들은 유튜브 영상의 자동 생성 자막처럼 정확도가 완벽하지는 않지만, 양이 엄청나게 많다는 특징을 가짐
    - 대량의 "약한" 데이터로 학습함으로써, Whisper는 다양한 언어, 배경 소음, 억양 등 현실 세계의 복잡한 음성 환경에 대한 높은 일반화 능력을 갖추게 됨

- transcribe : 음성 인식
    > model.transcribe(audio, fp16=False)
- translate : 번역  
    > model.translate(audio, fp16=False)

----

## **설치 순서(Windows)**

1. 가상환경 생성 (권장)
2. PyTorch 설치
3. Whisper 설치

### **1.가상환경 생성 (권장)**
터미널에서 실행한다.


In [ ]:
python -m venv whisper-env
whisper-env\Scripts\activate

- **만약 보안 오류가 발생한 경우(UnauthorizedAccess)**
- 해결 방법(영구해결)

1) VS Code 터미널(파워셸)에서 실행

In [ ]:
# 터미널에서 실행한다.
Set-ExecutionPolicy -Scope CurrentUser -ExecutionPolicy RemoteSigned

2) 새 터미널을 열고 가상환경 활성화

In [ ]:
# 터미널에서 실행한다.
.\whisper-env\Scripts\Activate.ps1

3) 확인

In [ ]:
# 터미널에서 실행한다.
python -c "import sys; print(sys.executable); print(sys.version)"

### **2.PyTorch 설치**

In [ ]:
pip install torch torchvision torchaudio

### **3.Whisper 설치**

In [ ]:
pip install git+https://github.com/openai/whisper.git

### **4.ffmpeg 설치**
- 참고 : https://angelplayer.tistory.com/351
- https://ffmpeg.org/  
- 다운로드 > 윈도우 버튼 >(Windows EXE Files) Windows builds from gyan.dev 선택
- ffmpeg-git-full.7z 선택 > 압축풀고 > 폴더 이름을 ffmpeg 으로 변경한다.
- ffmpeg 폴더 전체를 C:\로 이동시키고
- 고급 시스템 설정 > 환경변수 > 시스템 변수 > Path에 추가한다. (C:\ffmpeg\bin)
- 꼭!!! VSC 재시작한다.

In [ ]:
# 사전 점검 코드
from pathlib import Path
import shutil, sys

audio = Path(r"./오디오-슬라이드2.mp3")  # 필요시 절대경로로 바꾸세요
print("▶ CWD:", Path.cwd())
print("▶ Audio Path:", audio.resolve())
print("▶ Audio Exists?:", audio.exists())
print("▶ ffmpeg on PATH?:", shutil.which("ffmpeg"))
print("▶ ffprobe on PATH?:", shutil.which("ffprobe"))

▶ CWD: c:\python\ktcloud_genai
▶ Audio Path: C:\python\ktcloud_genai\오디오-슬라이드2.mp3
▶ Audio Exists?: True
▶ ffmpeg on PATH?: C:\ffmpeg\bin\ffmpeg.EXE
▶ ffprobe on PATH?: C:\ffmpeg\bin\ffprobe.EXE


In [ ]:
# - 터미널에서 실행
$env:Path = "C:\ffmpeg\bin;" + $env:Path
ffmpeg -version

In [ ]:
from pathlib import Path
print("CWD:", Path.cwd())
print("Rel exists?:", Path("./오디오-슬라이드2.mp3").exists())
print("Abs exists?:", Path(r"C:\python\ktcloud_genai\오디오-슬라이드2.mp3").exists())

CWD: c:\python\ktcloud_genai
Rel exists?: True
Abs exists?: True




---



## **Whisper로 음성 파일을 텍스트로 변환하기**

1. Whisper 라이브러리 불러오기
2. 모델 로드하기
3. 음성 파일을 읽어들여 변환
4. 결과 출력 또는 저장

- 영어 오디오

In [1]:
# 1.Whisper 라이브러리 불러오기
import whisper

# 2.모델 로드하기
model = whisper.load_model("small")  # tiny, base, small, medium, large

# 3.음성 파일을 읽어들여 변환 .wav, .mp3, .m4a, .webm, .ogg, .flac 등
audio = "아동-영어1.wav"
result = model.transcribe(audio, fp16=False)  # fp16=False 옵션은 CPU 환경에서 필요
print(result["text"])


# 4.결과 출력 또는 저장
with open("output_eng.txt", "w", encoding="utf-8") as f:
    f.write(result["text"])
    

 Pay attention to me.


- 한글 오디오

In [2]:
import whisper, shutil
from pathlib import Path


audio_path = Path("/Users/jangjihyeon/kt_cloud_tech_up/ktcloud_genai/오디오-슬라이드2.mp3") # 절대경로 권장


# 1) 필수 전제 확인
assert audio_path.exists(), f"오디오 파일이 없습니다: {audio_path}"
assert shutil.which("ffmpeg"), "ffmpeg를 찾을 수 없습니다. 설치 후 PATH에 등록하세요."
assert shutil.which("ffprobe"), "ffprobe를 찾을 수 없습니다. ffmpeg 설치 시 함께 등록됩니다."


# 2) 모델 로드 (CPU 사용 시 fp16 비활성화 권장)
model = whisper.load_model("small")   # tiny/base/small/medium/large
result = model.transcribe(str(audio_path), fp16=False)  # CPU면 fp16=False로 경고 억제
print(result["text"])


# 3) 저장
Path("output_kor.txt").write_text(result["text"], encoding="utf-8")
print("저장 완료: output_kor.txt")


 커피의 맛을 이해하기 위해서는 4가지 주요 요소를 살펴보는 것이 중요합니다. 첫째, 삼미는 심맛의 강도를 나타내며 이는 과일이나 로스팅 과정에서 기인합니다. 둘째, 바디감은 커피가 입 안에서 느껴지는 묵직함과 질감을 의미하며 커피의 무게감을 표현합니다.
저장 완료: output_kor.txt


----

## **실시간 음성 인식**

- 참고 : https://bcuts.tistory.com/197

- 반복 과정
    - 마이크로부터 5초간 음성
    - 입력 데이터를 .wav 파일로 저장
    - 저장된 파일을 Whisper로 처리
    - 텍스트 출력

### 라이브러리 설치

In [4]:
!brew install portaudio

==> Auto-updating Homebrew...
Adjust how often this is run with `$HOMEBREW_AUTO_UPDATE_SECS` or disable with
`$HOMEBREW_NO_AUTO_UPDATE=1`. Hide these hints with `$HOMEBREW_NO_ENV_HINTS=1` (see `man brew`).
==> Auto-updated Homebrew!
Updated 1 tap (homebrew/cask).

You have 18 outdated formulae installed.

==> Fetching downloads for: portaudio
==> Downloading https://ghcr.io/v2/homebrew/core/portaudio/manifests/19.7.0-1
######################################################################### 100.0%
==> Fetching portaudio
==> Downloading https://ghcr.io/v2/homebrew/core/portaudio/blobs/sha256:8ad9f1c1
######################################################################### 100.0%
==> Pouring portaudio--19.7.0.arm64_sequoia.bottle.1.tar.gz
🍺  /opt/homebrew/Cellar/portaudio/19.7.0: 34 files, 545.9KB
==> Running `brew cleanup portaudio`...
Disable this behaviour by setting `HOMEBREW_NO_INSTALL_CLEANUP=1`.
Hide these hints with `HOMEBREW_NO_ENV_HINTS=1` (see `man brew`).
==> No outdated de

In [5]:
!pip install pyaudio

  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyaudio: filename=pyaudio-0.2.14-cp313-cp313-macosx_11_0_arm64.whl size=24204 sha256=bc809680c7f6773780668ab0fbed0099657d0a3424527315b2c640ac3cd4b0df
  Stored in directory: /Users/jangjihyeon/Library/Caches/pip/wheels/32/45/57/aac45d8ad6f62e05779c15d0e62a09cfb51ff47c3fb1599b36
Successfully built pyaudio


In [ ]:
!pip install pyaudio numpy soundfile # 이렇게 하니까 에러가 뜬다

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.2 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [27 lines of output]
      /private/var/folders/j4/25jn9cwn74j37fv4yccqxp4w0000gn/T/pip-build-env-u44x6bsj/overlay/lib/python3.13/site-packages/setuptools/dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX license expression:
      
              License :: OSI Approved :: MIT License
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              ***********

### 실시간 음성인식 코드

In [3]:
import whisper
import pyaudio
import wave
import time

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "temp.wav"

model = whisper.load_model("base")
audio = pyaudio.PyAudio()

stream = audio.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK,
                    # 자동 시작 방지, 녹음 직전에 수동으로 start_stream() 호출
                    # 추론 중 녹음이 stop되지 않아 버퍼가 계속 쌓여 input overflowed 에러가 떠서 추가했음.
                    start = False 
                    )
# 주피터에선 안됨. .py 에서만 가능
print("실시간 음성 인식 시작 (Ctrl+C로 종료)")

try:
    while True:
        frames = []
        #수동으로 시작
        stream.start_stream()
        # 오디오 수집하는 구간
        for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
            data = stream.read(CHUNK) #청크단위로 데이터 수집
            frames.append(data) #청크데이터 frames에 append
        stream.stop_stream()
        wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb') #바이너리로 쓰기 (temp.wav 녹음파일 생성 혹은 덮어쓰기됨)
        
        # 녹음 환경 설정 (채널, 샘플폭, 샘플레이트 )
        wf.setnchannels(CHANNELS) # 채널 (1)
        wf.setsampwidth(audio.get_sample_size(FORMAT)) #샘플폭
        wf.setframerate(RATE) #샘플레이트 (16kHz)
        # 
        wf.writeframes(b''.join(frames)) #청크 묶고 파일에 기록
        wf.close() # 파일 닫기 (첨부터 with wave.open 쓰는게 더 나은듯?)

        result = model.transcribe(WAVE_OUTPUT_FILENAME) # 추론 (speech to text)
        # print(type(result)) # dict
        print("📢 인식된 텍스트:", result["text"])

except KeyboardInterrupt:
    print("종료합니다.")
    stream.stop_stream()
    stream.close()
    audio.terminate()
 


실시간 음성 인식 시작 (Ctrl+C로 종료)


/Users/jangjihyeon/miniconda3/lib/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


📢 인식된 텍스트:  다른��의


/Users/jangjihyeon/miniconda3/lib/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


📢 인식된 텍스트:  한국 super美國與 한국어


/Users/jangjihyeon/miniconda3/lib/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


📢 인식된 텍스트:  Hi.
종료합니다.


----